In [158]:
import requests
import pandas as pd
import statsmodels as sm
import statsmodels.api as sma
from scipy.stats import boxcox
import plotly.graph_objects as go
import plotly.subplots as sp
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# Загрузка данных

In [159]:
data_raw = requests.get('https://raw.githubusercontent.com/sikoraaxd/Homework/main/datasets/timeseries.csv').text
with open('data.csv', 'w') as f:
  f.write(data_raw)

In [160]:
df = pd.read_csv('data.csv', index_col=0, parse_dates=True)
df.dtypes

came    int64
dtype: object

In [161]:
df

,came
date,
2016-01-10,1345
2016-01-17,2066
2016-01-24,1979
2016-01-31,1909
2016-02-07,1575
...,...
2019-03-03,1483
2019-03-10,1172
2019-03-17,1435


In [162]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['came']))
fig.show()

# Проверяем гипотезу о нестационарности временного ряда с уровнем значимости p = 0.05

In [163]:
p_value = 0.05

test = sm.tsa.stattools.adfuller(df.came)
p_test = test[1]

if p_test < p_value:
  print('Ряд стационарен. Альтернативаня гипотеза не отвергнута')
else:
  print('Ряд нестационарен. Нулевая гипотеза не отвергнута')

Ряд нестационарен. Нулевая гипотеза не отвергнута


# Декомпозиция временного ряда: разбиение на тренд, сезонность, шум.


In [164]:
decomp = sm.tsa.seasonal.seasonal_decompose(df)

fig = sp.make_subplots(rows=4, cols=1)
fig.add_trace(go.Scatter(x=df.index, y=df['came'], name='Оригинальный ряд'), row=1, col=1)
fig.update_yaxes(title_text="Оригинальный ряд", row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=decomp.trend, name='Тренд'), row=2, col=1)
fig.update_yaxes(title_text="Тренд", row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=decomp.seasonal, name='Сезонность'), row=3, col=1)
fig.update_yaxes(title_text="Сезонность", row=3, col=1)
fig.add_trace(go.Scatter(x=df.index, y=decomp.resid, name='Шум'), row=4, col=1)
fig.update_yaxes(title_text="Шум", row=4, col=1)
fig.update_traces(showlegend=False)
fig.update_layout(title='Декомпозиция временного ряда', height=1000)
fig.show()

# Сезонное дифференцирование ряда в 12 месяцев

In [165]:
diff_series = df.diff().dropna()
seasonal_diff_series = df.diff(12).dropna()

fig = sp.make_subplots(rows=3, cols=1)
fig.add_trace(go.Scatter(x=df.index, y=df['came'], name='Оригинальный ряд'), row=1, col=1)
fig.update_yaxes(title_text="Оригинальный ряд", row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=diff_series['came'], name='Дифференцированный ряд'), row=2, col=1)
fig.update_yaxes(title_text="Дифференцированный ряд", row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=seasonal_diff_series['came'], name='Сезонно дифференцированный ряд'), row=3, col=1)
fig.update_yaxes(title_text="Сезонно дифференцированный ряд", row=3, col=1)

fig.update_traces(showlegend=False)
fig.update_layout(title='Дифференцирование временного ряда', height=1000)
fig.show()

#  Преобразование Бокса-Кокса

In [166]:
lamda = boxcox(df.came)[1]
print("Оптимальное значение lamda для преобразования: ", lamda)
df['boxcox_came'] = boxcox(df.came, lamda)
fig = sp.make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=df.index, y=df.came), row=1, col=1)
fig.update_yaxes(title_text="Оригинальный ряд", row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df.boxcox_came), row=2, col=1)
fig.update_yaxes(title_text="Преобразованнный ряд", row=2, col=1)
fig.update_traces(showlegend=False)
fig.update_layout(title='Преобразование Бокса-Кокса', height=1000)
fig.show()

Оптимальное значение lamda для преобразования:  0.11738125874670109


In [167]:
p_value = 0.05

test = sm.tsa.stattools.adfuller(df.boxcox_came)
p_test = test[1]

if p_test < p_value:
  print('Ряд стационарен. Альтернативаня гипотеза не отвергнута')
else:
  print('Ряд нестационарен. Нулевая гипотеза не отвергнута')

Ряд нестационарен. Нулевая гипотеза не отвергнута


# Обучение ARIMA и предсказание на 12 отсчётов вперёд

In [168]:


arima_order = sma.tsa.arma_order_select_ic(df.came, ic='aic')
p, q = arima_order['aic_min_order']

model = sma.tsa.ARIMA(df.came, order=(p, 0, q)).fit()

In [169]:
forecast = model.forecast(steps=12)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df.came, name='Оригинальный ряд'))
fig.add_trace(go.Scatter(x=forecast.index, y=forecast.values, name="Спрогнозированные данные", mode='lines', line=dict(color='red')))
fig.update_layout(title='Прогноз на 12 отсчётов вперёд')
fig.show()

# Разделение ряда на тренировочную и тестовую выборки. Обучение модели и оценка предсказаний

In [170]:
df = df.drop('boxcox_came', axis=1)
train = df.loc['2016-01-10':'2018-03-04']
test = df.loc['2018-03-11':]

arima_order = sma.tsa.arma_order_select_ic(train.came, ic='aic', trend='n')
p, q = arima_order['aic_min_order']

model = sma.tsa.ARIMA(train.came, order=(p, 0, q)).fit()

forecast = model.forecast(steps=len(test))

In [171]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train.came, name='Тренировочный ряд'))
fig.add_trace(go.Scatter(x=forecast.index, y=forecast.values, name="Спрогнозированные данные", mode='lines', line=dict(color='red')))
fig.update_layout(title='Прогноз временного ряда')
fig.show()

In [172]:
metrics = {
    'mae': np.mean(np.abs(forecast.values - test.came)),
    'mape': np.mean(np.abs((test.came - forecast.values) / test.came)) * 100,
    'smape': np.mean(2 * np.abs(forecast.values - test.came) / (np.abs(test.came) + np.abs(forecast.values))) * 100
}

metrics_df = pd.DataFrame(metrics, index=['Значение'])
metrics_df

,mae,mape,smape
Значение,296.076094,21.841788,19.090922
